# TP Machine Learning - Industrie Sidérurgique
## Partie 1 : Exploration des données

### Introduction au dataset

Ce TP utilise un jeu de données provenant d'une industrie sidérurgique, collecté sur une période de 2018 à 2019.
Il contient des mesures énergétiques détaillées d'une installation industrielle, offrant une opportunité unique
d'analyser et d'optimiser la consommation énergétique dans un contexte industriel réel.

#### Description des variables :

1. **Variables temporelles :**
   - `date` : Date et heure de la mesure
   - `Day_of_week` : Jour de la semaine (Monday à Sunday)
   - `NSM` : Number of Seconds from Midnight (nombre de secondes depuis minuit)
   - `WeekStatus` : Type de jour (Weekday/Weekend)

2. **Variables énergétiques principales :**
   - `Usage_kWh` : Consommation d'énergie en kilowattheures (TARGET)
   - `Lagging_Current_Reactive.Power_kVarh` : Puissance réactive en retard
   - `Leading_Current_Reactive_Power_kVarh` : Puissance réactive en avance
   - `CO2(tCO2)` : Émissions de CO2 en tonnes

3. **Facteurs de puissance :**
   - `Lagging_Current_Power_Factor` : Facteur de puissance en retard
   - `Leading_Current_Power_Factor` : Facteur de puissance en avance

#### Applications possibles :

1. **Prédiction de consommation :**
   - Prévision de la consommation énergétique
   - Estimation des émissions de CO2
   - Planification de la production

2. **Optimisation énergétique :**
   - Identification des périodes de forte consommation
   - Analyse de l'efficacité énergétique
   - Réduction des émissions de CO2

3. **Détection d'anomalies :**
   - Identification des consommations inhabituelles
   - Détection des dysfonctionnements
   - Maintenance prédictive

4. **Analyse des patterns :**
   - Variations journalières et hebdomadaires
   - Impact des jours ouvrés vs weekends
   - Corrélations entre variables énergétiques

In [ ]:
# Import des packages nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import normaltest

# Configuration de l'affichage
sns.set_theme()
%matplotlib inline

# Téléchargement et chargement des données
!wget -O steel_industry_data.zip https://archive.ics.uci.edu/static/public/851/steel+industry+energy+consumption.zip
!unzip -o steel_industry_data.zip

# Chargement des données
df = pd.read_csv('Steel_industry_data.csv')

# Conversion des dates avec format européen (jour/mois/année)
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

# Affichage des premières lignes avec une meilleure présentation
print("\nAperçu des premières lignes :")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(df.head().to_string())

# Information sur la structure du dataset
print("\nStructure du dataset :")
print(f"Nombre d'observations : {df.shape[0]:,}")
print(f"Nombre de variables : {df.shape[1]:,}")

# Résumé des types de variables
print("\nTypes de variables :")
display(df.dtypes)

# Exemple de valeurs pour les variables catégorielles
print("\nValeurs uniques dans les variables catégorielles :")
for col in ['Day_of_week', 'WeekStatus', 'Load_Type']:
    print(f"\n{col} :")
    print(df[col].value_counts())

# Statistiques descriptives détaillées
print("\nStatistiques descriptives des variables numériques :")
# /!\ Complétez les '...' pour obtenir une description (Pandas) du dataset /!\
desc_stats = ...
display(desc_stats)

# Vérification de la couverture temporelle
print("\nPériode couverte par le dataset :")
print(f"Début : {df['date'].min()}")
print(f"Fin : {df['date'].max()}")
print(f"Durée : {(df['date'].max() - df['date'].min()).days} jours")

# Vérification des valeurs manquantes
print("\nValeurs manquantes par variable :")
print(df.isnull().sum())

### Points d'attention pour l'analyse :

1. **Prétraitement nécessaire :**
   - Standardisation des variables numériques
   - Encodage des variables catégorielles
   - Gestion de la temporalité

2. **Aspects métier à considérer :**
   - Cycles de production industrielle
   - Contraintes énergétiques
   - Objectifs environnementaux (CO2)

3. **Opportunités d'analyse :**
   - Patterns de consommation
   - Efficacité énergétique
   - Optimisation des coûts

In [ ]:
# Visualisation initiale des distributions
plt.figure(figsize=(15, 10))

# Distribution de la consommation énergétique
plt.subplot(2, 2, 1)
sns.histplot(data=df, x='Usage_kWh', bins=50)
plt.title('Distribution de la consommation énergétique')

# Consommation par jour de la semaine
plt.subplot(2, 2, 2)
sns.boxplot(data=df, x='Day_of_week', y='Usage_kWh')
# /!\ Complétez les '...' pour orienter les labels d'abscisse avec une roration de 45° (Matplotlib) /!\
plt.xticks(...)
plt.title('Consommation par jour')

# Évolution temporelle
plt.subplot(2, 2, 3)
df.set_index('date')['Usage_kWh'].plot()
plt.title('Évolution de la consommation dans le temps')

# Relation CO2/Consommation
plt.subplot(2, 2, 4)
plt.scatter(df['Usage_kWh'], df['CO2(tCO2)'], alpha=0.5)
plt.xlabel('Consommation (kWh)')
plt.ylabel('Émissions CO2 (tCO2)')
plt.title('Relation Consommation/Émissions')

plt.tight_layout()
plt.show()

❓ **Questions :**
1. Combien de variables numériques et catégorielles avons-nous ?
2. Y a-t-il des valeurs manquantes à traiter ?
3. Quelles sont les plages de valeurs pour chaque variable ?
4. Quelles sont les principales caractéristiques de la consommation énergétique ?
5. Comment la consommation varie-t-elle selon les jours de la semaine ?
6. Quelle est la nature de la relation entre consommation et émissions CO2 ?

### 2. Analyse des distributions
Visualisons la distribution de nos variables principales.

In [ ]:
# Visualisation de la distribution des variables numériques
plt.figure(figsize=(15, 10))
df.select_dtypes(include=['float64']).hist(bins=30)
plt.tight_layout()
plt.show()

### 3. Analyse des corrélations
Étudions les relations entre nos variables.

In [ ]:
# Prétraitement des données pour la corrélation
# Suppression des colonnes non numériques
df_num = df.select_dtypes(include=['float64', 'int64'])

# Matrice de corrélation
plt.figure(figsize=(12, 8))
sns.heatmap(df_num.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de corrélation')
plt.show()

# Scatter matrix pour toutes les variables numériques
# Configuration de la taille de police pour les labels
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6

# Création du scatter matrix
axes = pd.plotting.scatter_matrix(df_num,
                                figsize=(8, 8),
                                diagonal='kde',
                                alpha=0.5,
                                density_kwds={'alpha': 0.2},
                                marker='.',
                                s=20)  # Taille des points réduite

# Rotation des labels pour une meilleure lisibilité
for ax in axes.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

# Réinitialisation des paramètres de police
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

❓ **Questions :**
1. Quelles variables sont les plus corrélées avec la consommation d'énergie ?
2. Observez-vous des corrélations surprenantes ?
3. Quelles variables semblent les moins importantes ?

### 4. Analyse approfondie des relations entre variables

Jusqu'ici, nous avons analysé les corrélations linéaires (Pearson) entre nos variables.
Cependant, dans les données réelles, les relations peuvent être plus complexes.
Approfondissons notre analyse en trois étapes :

1. **Création de nouveaux indicateurs** : Ratios pertinents pour l'analyse énergétique
2. **Analyse des corrélations non linéaires** : Utilisation du coefficient de Spearman
3. **Visualisation des interactions** : Impact des différents facteurs sur la consommation

#### 4.1 Création d'indicateurs énergétiques

In [ ]:
# Création de ratios énergétiques pertinents
df['power_factor_ratio'] = df['Lagging_Current_Power_Factor'] / df['Leading_Current_Power_Factor']
df['reactive_power_ratio'] = df['Lagging_Current_Reactive.Power_kVarh'] / df['Leading_Current_Reactive_Power_kVarh']

#### 4.2 Comparaison des corrélations linéaires et non linéaires

- **Corrélation de Pearson** (vue précédemment) : mesure les relations linéaires
- **Corrélation de Spearman** : mesure les relations monotones (même non linéaires)

Comparons les deux approches :

In [ ]:
# Sélection des colonnes numériques
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Création des deux matrices de corrélation
pearson_corr = df[numeric_cols].corr(method='pearson')
# /!\ Complétez les '...' pour obtenir une matrice de corrélation avec la méthode Spearman /!\
spearman_corr = df[numeric_cols]...

# Visualisation côte à côte
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', fmt='.2f', ax=ax1)
ax1.set_title('Corrélations de Pearson\n(relations linéaires)')

sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt='.2f', ax=ax2)
ax2.set_title('Corrélations de Spearman\n(relations monotones)')

plt.tight_layout()
plt.show()

#### 4.3 Analyse des interactions spécifiques

Visualisons quelques relations importantes pour comprendre leur nature :

In [ ]:
# Visualisation des interactions clés
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.scatterplot(data=df, x='power_factor_ratio', y='Usage_kWh', alpha=0.5)
plt.title('Consommation vs Ratio des facteurs de puissance')

plt.subplot(1, 3, 2)
# /!\ Complétez les '...' pour obtenir un scatterplot (Seaborn) du reactive power ratio en fonction de la consommation énérgetique /!\
sns.scatterplot(...)
plt.title('Consommation vs Ratio des puissances réactives')

plt.subplot(1, 3, 3)
sns.scatterplot(data=df, x='CO2(tCO2)', y='Usage_kWh',
                hue='WeekStatus', alpha=0.5)
plt.title('Consommation vs CO2 par type de jour')

plt.tight_layout()
plt.show()

❓ **Questions d'analyse :**

1. **Comparaison des corrélations**
   - Quelles différences observez-vous entre les corrélations de Pearson et de Spearman ?
   - Pour quelles variables les différences sont-elles les plus marquées ?
   - Que nous apprend cela sur la nature des relations entre variables ?

2. **Ratios énergétiques**
   - Pourquoi avoir créé ces ratios spécifiques ?
   - Que révèlent-ils sur l'efficacité énergétique ?

3. **Patterns de consommation**
   - Comment la relation CO2/consommation varie-t-elle selon le type de jour ?
   - Quelles implications cela a-t-il pour la gestion énergétique ?
   - Quelles recommandations pourriez-vous faire ?

### 5. Préparation des données pour l'apprentissage

In [ ]:
# Préparation des données pour les tests
# Conversion des variables catégorielles
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Encodage des variables catégorielles
# Définir l'ordre explicite des jours
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Encodage des variables catégorielles
categorical_features = ['Day_of_week', 'WeekStatus']
encoder = OneHotEncoder(sparse_output=False)  # Enlever drop='first' pour voir tous les jours

# S'assurer que les jours sont dans le bon ordre
df['Day_of_week'] = pd.Categorical(df['Day_of_week'], categories=days_order, ordered=True)

# Encoder les variables
encoded_features = encoder.fit_transform(df[categorical_features])

# Création des noms de colonnes pour les variables encodées
day_names = [f'Day_{day}' for day in encoder.categories_[0]]  # Tous les jours
week_status_names = [f'Status_{status}' for status in encoder.categories_[1]]
encoded_columns = day_names + week_status_names

# Afficher pour vérification
print("Catégories des jours :", encoder.categories_[0])
print("Jours encodés :", day_names)

# Création du DataFrame avec les variables encodées
df_encoded = pd.DataFrame(encoded_features, columns=encoded_columns)

# Sélection des features numériques
numeric_features = [
    'Usage_kWh',
    'Lagging_Current_Reactive.Power_kVarh',
    'Leading_Current_Reactive_Power_kVarh',
    'CO2(tCO2)',
    'Lagging_Current_Power_Factor',
    'Leading_Current_Power_Factor',
    'NSM'
]

# Standardisation des variables numériques
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df[numeric_features]),
    columns=numeric_features
)

# Combinaison des features numériques et encodées
# /!\ Complétez les '...' pour obtenir une concaténation (Pandas) du df_scaled et df_encoded /!\
df_final = ...

display(df_final)
# display(df_final.loc[2000:2300])

# Vérification des corrélations résiduelles
plt.figure(figsize=(12, 8))
sns.heatmap(df_final.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Corrélations des features préparées')
plt.show()

❓ **Questions :**

1. **Relations entre variables énergétiques**
   - Quelle est la relation entre la consommation (Usage_kWh) et les émissions CO2 ?
   - Pourquoi observe-t-on une forte corrélation entre ces variables ?
   - Quelles autres variables sont fortement corrélées à la consommation ?

2. **Relations entre facteurs de puissance**
   - Comment interpréter la corrélation entre Lagging et Leading Power Factor ?
   - Pourquoi ces facteurs ont-ils des relations différentes avec la consommation ?
   - Quel impact cela peut-il avoir sur l'efficacité énergétique ?

3. **Structure des données**
   - Y a-t-il des variables redondantes qu'on pourrait éliminer ?
   - Quelles variables semblent les plus importantes pour la prédiction ?

4. **Implications pratiques**
   - Comment ces corrélations peuvent-elles guider l'optimisation énergétique ?
   - Quelles variables devraient être surveillées en priorité ?
   - Quelles recommandations business peut-on en tirer ?

### 6. Analyse temporelle approfondie

L'analyse temporelle est cruciale pour comprendre les patterns de consommation énergétique.

In [ ]:
# Conversion de la colonne NSM (Number of Seconds from Midnight) en heure
df['hour'] = df['NSM'] / 3600

# Analyse horaire
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
hourly_consumption = df.groupby('hour')['Usage_kWh'].mean()
plt.plot(hourly_consumption.index, hourly_consumption.values)
plt.title('Consommation moyenne par heure')
plt.xlabel('Heure')
plt.ylabel('Consommation (kWh)')

plt.subplot(1, 2, 2)
sns.boxplot(data=df, x='Day_of_week', y='Usage_kWh')
plt.title('Distribution de la consommation par jour')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Analyse des patterns hebdomadaires
weekly_stats = df.groupby('Day_of_week').agg({
    'Usage_kWh': ['mean', 'std', 'min', 'max'],
    'CO2(tCO2)': ['mean', 'std']
}).round(2)

print("\nStatistiques hebdomadaires :")
display(weekly_stats)

❓ **Questions :**
1. Quelles sont les heures de pic de consommation ?
2. Y a-t-il une différence significative entre les jours de la semaine ?
3. Comment expliquer les variations observées ?